<a href="https://colab.research.google.com/github/szl10/Human_Activity_Recognition/blob/main/HAR_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [551]:
import pandas as pd
import numpy as np
import os

In [552]:
#Activities are the class label
Activities={
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING'
}

In [553]:
# Utility function to print confusiion matrix
def confusion_matrix(Y_true,Y_pred):
  Y_true=pd.Series([Activities[y] for y in np.argmax(Y_true,axis=1)])
  Y_pred=pd.Series([Activities[y] for y in np.argmax(Y_pred,axis=1)])

  return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

# DATA

In [554]:
#Data Directory
DATADIR=f'{os.getcwd()}/UCI_HAR_Dataset'

In [555]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration

SIGNALS = [
    'body_acc_x',
    'body_acc_y',
    'body_acc_z',
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

# Function to read data from csv files

In [556]:
# Utility function to read the data from csv file
def _read_csv(filename):
  return pd.read_csv(filename,delim_whitespace=True,header=None)

# Utility function to load the load
def load_signals(subset):
  signals_data = []

  for signal in SIGNALS:
    # filename= f'{DATADIR}/{subset}/Inertial_Signals/{signal}_{subset}.txt'
    # /content/UCI_HAR_Dataset/test/Intertial_Signals/body_acc_x_test.txt
    filename = '/content/UCI_HAR_Dataset/' + subset + '/Inertial_Signals/' + signal + '_' + subset + '.txt'
    signals_data.append(
        _read_csv(filename).to_numpy()
      )


    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
  return np.transpose(signals_data, (1, 2, 0))


In [557]:
def load_y(subset):
    filename = '/content/UCI_HAR_Dataset/' + subset + '/y_' + subset + '.txt'
    y = _read_csv(filename)[0]
    return pd.get_dummies(y).to_numpy()


In [558]:
def load_data():
  X_train,X_test = load_signals('train'),load_signals('test')
  Y_train,Y_test = load_y('train'),load_y('test')

  return X_train,X_test,Y_train,Y_test

In [559]:
np.random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)

In [560]:
# Configuring a session
session_conf = tf.compat.v1.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [561]:
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

In [562]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout



In [563]:
#Initialising the Parameters
epochs=30
batch_size=16
n_hidden=32

In [564]:
#utility function to count the number of classes
def _count_classes(y):
  return len(set([tuple(category) for category in y]))

In [565]:
X_train,X_test,Y_train,Y_test=load_data()


In [566]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(7352, 128, 9)
(7352, 6)
(2947, 128, 9)
(2947, 6)


In [567]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

In [568]:
print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


In [569]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 32)                5376      
                                                                 
 dropout_8 (Dropout)         (None, 32)                0         
                                                                 
 dense_8 (Dense)             (None, 6)                 198       
                                                                 
Total params: 5574 (21.77 KB)
Trainable params: 5574 (21.77 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [570]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [547]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Epoch 1/30
460/460 [==============================] - 37s 69ms/step - loss: 1.1073 - accuracy: 0.5305 - val_loss: 0.8451 - val_accuracy: 0.6267
Epoch 2/30
460/460 [==============================] - 26s 56ms/step - loss: 0.7704 - accuracy: 0.6487 - val_loss: 0.7589 - val_accuracy: 0.6647
Epoch 3/30
460/460 [==============================] - 25s 55ms/step - loss: 0.6848 - accuracy: 0.7014 - val_loss: 0.7616 - val_accuracy: 0.6939
Epoch 4/30
460/460 [==============================] - 26s 57ms/step - loss: 0.5842 - accuracy: 0.7625 - val_loss: 0.6771 - val_accuracy: 0.7357
Epoch 5/30
460/460 [==============================] - 26s 57ms/step - loss: 0.5123 - accuracy: 0.8215 - val_loss: 0.7353 - val_accuracy: 0.7615
Epoch 6/30
460/460 [==============================] - 25s 54ms/step - loss: 0.3897 - accuracy: 0.8768 - val_loss: 0.5460 - val_accuracy: 0.8388
Epoch 7/30
460/460 [==============================] - 25s 54ms/step - loss: 0.2988 - accuracy: 0.9094 - val_loss: 0.4432 - val_accuracy:

In [548]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

93/93 [==============================] - 2s 17ms/step
Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 509        0        27        0                   1   
SITTING                  0      404        84        0                   1   
STANDING                 0       89       442        0                   0   
WALKING                  0        0         0      455                  10   
WALKING_DOWNSTAIRS       0        1         0        0                 406   
WALKING_UPSTAIRS         0       12         0        2                   0   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            2  
STANDING                           1  
WALKING                           31  
WALKING_DOWNSTAIRS                13  
WALKING_UPSTAIRS                 457  


In [549]:
score = model.evaluate(X_test, Y_test)

93/93 [==============================] - 1s 13ms/step - loss: 0.4753 - accuracy: 0.9070


In [550]:
score

[0.47528010606765747, 0.907024085521698]